In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [22]:
drugall = pd.read_csv('../../big datasets/drug_data_to_use.csv')

In [27]:
drug_neg = drugall.iloc[1017:,:].copy()
classtox = []

for index, row in drug_neg.iterrows():
    classtox.append(0)
    
drug_neg.insert(loc=1, column='Class', value=classtox)



drug_pos = drugall.iloc[:1017,:].copy()
classdrug = []

for index, row in drug_pos.iterrows():
    classdrug.append(1)
    
drug_pos.insert(loc=1, column='Class', value=classdrug)

drug_pos.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1, inplace=True)
drug_neg.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1, inplace=True)
drug_pos.head()

,Class,nAcid,ALogP,ALogp2,AMR,nN,nO,nS,nP,nF,...,JGI7,JGI8,JGI9,JGI10,VE1_D,VR1_D,WTPT-1,WTPT-2,WTPT-3,XLogP
0,1,0.0,0.2842,0.080770,86.9785,0.0,4.0,0.0,0.0,0.0,...,0.021016,0.016358,0.012000,0.012346,0.009428,230.520837,47.495419,2.065018,11.316324,2.665
1,1,0.0,-0.6801,0.462536,94.5871,2.0,2.0,0.0,0.0,0.0,...,0.011177,0.018664,0.007760,0.010000,0.163765,250.398458,47.828913,2.079518,13.788202,0.939
2,1,0.0,1.0460,1.094116,87.4012,0.0,4.0,0.0,0.0,0.0,...,0.019206,0.008902,0.010859,0.013740,0.124083,1645.934186,43.058016,2.050382,11.246419,2.072
3,1,0.0,-0.3414,0.116554,64.9254,0.0,7.0,0.0,0.0,0.0,...,0.020991,0.008724,0.013333,0.000000,0.169516,179.576794,36.989542,1.946818,18.859169,1.052
4,1,0.0,1.6169,2.614366,70.6318,1.0,4.0,0.0,0.0,0.0,...,0.024706,0.013454,0.009896,0.012346,0.147100,143.314817,36.802051,1.936950,13.302624,1.450


In [28]:
drugall = drug_pos.append(drug_neg)

In [29]:
drugall.to_csv('../../big datasets/drugall_with_classes.csv')

In [43]:
drug_pos_features = drug_pos.iloc[:,1:]
drug_pos_features.head()
drug_pos_class = drug_pos['Class']

drug_neg_features = drug_neg.iloc[:,1:]
drug_neg_class = drug_neg['Class']

In [44]:
x_train_neg, x_test_neg, y_train_neg, y_test_neg = train_test_split(drug_neg_features, drug_neg_class, test_size=0.2, random_state=12)
x_train, x_test, y_train, y_test = train_test_split(drug_pos_features, drug_pos_class, test_size=0.2, random_state=12)


In [45]:
x_train = x_train.append(x_train_neg)
x_test = x_test.append(x_test_neg)
y_train = y_train.append(y_train_neg)
y_test = y_test.append(y_test_neg)
all_train = x_train.join(y_train)
all_test = x_test.join(y_test)
len(y_test) == len(x_test)
len(x_train) == len(y_train)

True

In [48]:
all_train.drop(all_train.select_dtypes(['object']), inplace=True, axis=1)
all_train.astype('float64')

all_test.drop(all_test.select_dtypes(['object']), inplace=True, axis=1)
all_test.astype('float64')

all_train.head(), all_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 372 to 1172
Columns: 279 entries, nAcid to Class
dtypes: float64(278), int64(1)
memory usage: 2.3 MB


(     nAcid   ALogP    ALogp2       AMR   nN   nO   nS   nP   nF  nCl  ...  \
 372    0.0 -2.0083  4.033269   83.5689  2.0  5.0  0.0  0.0  1.0  0.0  ...   
 360    0.0 -1.5431  2.381158   98.3173  3.0  4.0  1.0  0.0  0.0  0.0  ...   
 799    0.0  0.1530  0.023409   98.4411  2.0  3.0  0.0  0.0  0.0  0.0  ...   
 263    0.0 -3.0456  9.275679  102.8853  5.0  1.0  0.0  0.0  0.0  0.0  ...   
 425    0.0 -0.5105  0.260610   93.8320  2.0  3.0  0.0  0.0  0.0  0.0  ...   
 
          JGI8      JGI9     JGI10     VE1_D       VR1_D     WTPT-1    WTPT-2  \
 372  0.011339  0.009392  0.005648  0.004290  252.735571  49.913289  1.996532   
 360  0.013213  0.009712  0.008328  0.158715  300.918715  52.817771  2.031453   
 799  0.008273  0.009643  0.005663  0.066221  368.184630  49.516759  2.063198   
 263  0.005603  0.008608  0.005493  0.083425  344.368465  55.858018  2.068815   
 425  0.019596  0.013438  0.013555  0.230550  200.220486  49.422468  2.059269   
 
         WTPT-3  XLogP  Class  
 372  21.7

In [49]:
with pd.option_context('mode.use_inf_as_null', True):
    all_train = all_train.dropna(how='all', axis=1)
    all_test = all_test.dropna(how='all',axis=1)

In [50]:
all_train = all_train[np.isfinite(all_train).all(1)]
all_test = all_test[np.isfinite(all_test).all(1)]
all_train.shape[1] == all_test.shape[1]

True

In [51]:
y_train = all_train['Class']
x_train = all_train.drop(['Class'], axis=1)

x_test = all_test.drop(['Class'], axis=1)
y_test = all_test['Class']

In [52]:
y_train.tail(100)

1164    0
1269    0
1043    0
1208    0
1096    0
1039    0
1265    0
1089    0
1093    0
1155    0
1063    0
1181    0
1288    0
1071    0
1261    0
1088    0
1180    0
1197    0
1206    0
1193    0
1019    0
1045    0
1304    0
1026    0
1086    0
1264    0
1170    0
1114    0
1119    0
1201    0
       ..
1182    0
1061    0
1082    0
1241    0
1021    0
1242    0
1263    0
1042    0
1275    0
1044    0
1177    0
1257    0
1090    0
1126    0
1140    0
1151    0
1163    0
1136    0
1175    0
1117    0
1221    0
1091    0
1135    0
1158    0
1295    0
1276    0
1147    0
1258    0
1270    0
1172    0
Name: Class, Length: 100, dtype: int64

In [53]:
from sklearn.preprocessing import StandardScaler
xscaler = StandardScaler().fit(x_train)
x_train = xscaler.transform(x_train)
testscaler = StandardScaler().fit(x_test)
x_test = testscaler.transform(x_test)

In [54]:
sm = SMOTE(random_state=2)
x_train_res, y_train_res = sm.fit_resample(x_train, y_train)

In [55]:
print(len(x_train),len(y_train),len(x_train_res),len(y_train_res))

990 990 1626 1626


In [17]:
x_train_res = pd.DataFrame(x_train_res)
y_train_res = pd.DataFrame(y_train_res)
x_test = pd.DataFrame(x_test)
y_test = pd.DataFrame(y_test)

In [18]:
x_train_res.to_csv('../../big datasets/drugml/x_train_res.csv')
y_train_res.to_csv('../../big datasets/drugml/y_train_res.csv')
x_test.to_csv('../../big datasets/drugml/x_test.csv')
y_test.to_csv('../../big datasets/drugml/y_test.csv')

In [56]:
y_train.tail(20)

1177    0
1257    0
1090    0
1126    0
1140    0
1151    0
1163    0
1136    0
1175    0
1117    0
1221    0
1091    0
1135    0
1158    0
1295    0
1276    0
1147    0
1258    0
1270    0
1172    0
Name: Class, dtype: int64

In [20]:
y_train.describe()

count    990.000000
mean       0.821212
std        0.383368
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Class, dtype: float64